In [ ]:
# if play the code in CoLab
import pandas as pd
import seaborn as sns
from google.colab import files
import io
import warnings

warnings.filterwarnings(action='ignore')

In [ ]:
# if play the code in CoLab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# if play the code in CoLab
%cd drive/My Drive/Colab Notebooks/인지프/프로젝트/ProTACT/tensorflow

In [1]:
# Load autoreload extension
%load_ext autoreload

# Set autoreload behavior
%autoreload 2
import os
import time
import argparse
import random
import numpy as np
from models.ProTACT import build_ProTACT
import tensorflow as tf
from configs.configs import Configs
from utils.read_data_pr import read_pos_vocab, read_word_vocab, read_prompts_we, read_essays_prompts, read_prompts_pos
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks, load_word_embedding_dict, build_embedd_table
from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator
from tensorflow import keras
import matplotlib.pyplot as plt
    

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/nih1226/miniforge3/envs/protact/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3250998/1392874311.py", line 11, in <module>
    from models.ProTACT import build_ProTACT
  File "/home/nih1226/ai_project/ProTACT/tensorflow/models/ProTACT.py", line 2, in <module>
    import tensorflow as tf
  File "/home/nih1226/miniforge3/envs/protact/lib/python3.7/site-packages/tensorflow/__init__.py", line 98, in <module>
    from tensorflow_core import *
  File "/home/nih1226/miniforge3/envs/protact/lib/python3.7/site-packages/tensorflow_core/__init__.py", line 40, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 959, in _find_and_load_unlocked
  File "/home/nih1226/miniforge3/envs/protact/lib/pyth

In [4]:
# parser = argparse.ArgumentParser(description="ProTACT model")
# parser.add_argument('--test_prompt_id', type=int, default=1, help='prompt id of test essay set')
# parser.add_argument('--seed', type=int, default=12, help='set random seed')
# parser.add_argument('--model_name', type=str,
#                     choices=['ProTACT'],
#                     help='name of model')
# parser.add_argument('--num_heads', type=int, default=2, help='set the number of heads in Multihead Attention')
# parser.add_argument('--features_path', type=str, default='data/hand_crafted_v3.csv')

test_prompt_id = 1
seed = 1
num_heads = 2
features_path = '../data/hand_crafted_v3.csv'

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

print("Test prompt id is {} of type {}".format(test_prompt_id, type(test_prompt_id)))
print("Seed: {}".format(seed))

configs = Configs()

data_path = configs.DATA_PATH
train_path = data_path + str(test_prompt_id) + '/train.pk'
dev_path = data_path + str(test_prompt_id) + '/dev.pk'
test_path = data_path + str(test_prompt_id) + '/test.pk'
pretrained_embedding = configs.PRETRAINED_EMBEDDING
embedding_path = configs.EMBEDDING_PATH
readability_path = configs.READABILITY_PATH
prompt_path = configs.PROMPT_PATH
vocab_size = configs.VOCAB_SIZE
epochs = configs.EPOCHS
batch_size = configs.BATCH_SIZE
print("Numhead : ", num_heads, " | Features : ", features_path, " | Pos_emb : ", configs.EMBEDDING_DIM)

read_configs = {
    'train_path': train_path,
    'dev_path': dev_path,
    'test_path': test_path,
    'features_path': features_path,
    'readability_path': readability_path,
    'vocab_size': vocab_size
}

Test prompt id is 1 of type <class 'int'>
Seed: 1
Numhead :  2  | Features :  ../data/hand_crafted_v3.csv  | Pos_emb :  50


In [5]:
# read POS for prompts
pos_vocab = read_pos_vocab(read_configs)
prompt_pos_data = read_prompts_pos(prompt_path, pos_vocab) # for prompt POS embedding 

# read words for prompts 
word_vocab = read_word_vocab(read_configs)
prompt_data = read_prompts_we(prompt_path, word_vocab) # for prompt word embedding 

# read essays and prompts
train_data, dev_data, test_data = read_essays_prompts(read_configs, prompt_data, prompt_pos_data, pos_vocab) 

if pretrained_embedding:
    embedd_dict, embedd_dim, _ = load_word_embedding_dict(embedding_path)
    embedd_matrix = build_embedd_table(word_vocab, embedd_dict, embedd_dim, caseless=True)
    embed_table = [embedd_matrix]
else:
    embed_table = None

 prompt_pos size: 8
 prompt_words size: 8
 pos_x size: 9513
 readability_x size: 9513
 pos_x size: 1680
 readability_x size: 1680
 pos_x size: 1783
 readability_x size: 1783
Loading GloVe ...
OOV number =189, OOV ratio = 0.047262


In [ ]:
def sample_data_dict(data_dict, sample_ratio=0.1):
    keys = list(data_dict.keys())
    # 먼저 첫 번째 키에 대한 샘플 크기를 계산합니다.
    sample_size = int(len(data_dict[keys[0]]) * sample_ratio)
    indices = np.random.choice(len(data_dict[keys[0]]), sample_size, replace=False)

    sampled_dict = {}
    for key in keys:
        if isinstance(data_dict[key], list) or isinstance(data_dict[key], np.ndarray):
            sampled_dict[key] = [data_dict[key][i] for i in indices]
        else:
            sampled_dict[key] = data_dict[key]

    return sampled_dict
# train data만 10% 사용. 나머지는 비교를 위해 그대로 사용한다.
train_data = sample_data_dict(train_data) # train data를 10%만 사용
#dev_data = sample_data_dict(dev_data) # dev data를 10%만 사용
#test_data = sample_data_dict(test_data) # test data를 10%만 사용

In [6]:
max_sentlen = max(train_data['max_sentlen'], dev_data['max_sentlen'], test_data['max_sentlen'])
max_sentnum = max(train_data['max_sentnum'], dev_data['max_sentnum'], test_data['max_sentnum'])
prompt_max_sentlen = prompt_data['max_sentlen']
prompt_max_sentnum = prompt_data['max_sentnum']

print('max sent length: {}'.format(max_sentlen))
print('max sent num: {}'.format(max_sentnum))
print('max prompt sent length: {}'.format(prompt_max_sentlen))
print('max prompt sent num: {}'.format(prompt_max_sentnum))

train_data['y_scaled'] = get_scaled_down_scores(train_data['data_y'], train_data['prompt_ids'])
dev_data['y_scaled'] = get_scaled_down_scores(dev_data['data_y'], dev_data['prompt_ids'])
test_data['y_scaled'] = get_scaled_down_scores(test_data['data_y'], test_data['prompt_ids'])

X_train_pos = pad_hierarchical_text_sequences(train_data['pos_x'], max_sentnum, max_sentlen)
X_dev_pos = pad_hierarchical_text_sequences(dev_data['pos_x'], max_sentnum, max_sentlen)
X_test_pos = pad_hierarchical_text_sequences(test_data['pos_x'], max_sentnum, max_sentlen)

X_train_pos = X_train_pos.reshape((X_train_pos.shape[0], X_train_pos.shape[1] * X_train_pos.shape[2]))
X_dev_pos = X_dev_pos.reshape((X_dev_pos.shape[0], X_dev_pos.shape[1] * X_dev_pos.shape[2]))
X_test_pos = X_test_pos.reshape((X_test_pos.shape[0], X_test_pos.shape[1] * X_test_pos.shape[2]))

X_train_prompt = pad_hierarchical_text_sequences(train_data['prompt_words'], max_sentnum, max_sentlen)
X_dev_prompt = pad_hierarchical_text_sequences(dev_data['prompt_words'], max_sentnum, max_sentlen)
X_test_prompt = pad_hierarchical_text_sequences(test_data['prompt_words'], max_sentnum, max_sentlen)

X_train_prompt = X_train_prompt.reshape((X_train_prompt.shape[0], X_train_prompt.shape[1] * X_train_prompt.shape[2]))
X_dev_prompt = X_dev_prompt.reshape((X_dev_prompt.shape[0], X_dev_prompt.shape[1] * X_dev_prompt.shape[2]))
X_test_prompt = X_test_prompt.reshape((X_test_prompt.shape[0], X_test_prompt.shape[1] * X_test_prompt.shape[2]))

X_train_prompt_pos = pad_hierarchical_text_sequences(train_data['prompt_pos'], max_sentnum, max_sentlen)
X_dev_prompt_pos = pad_hierarchical_text_sequences(dev_data['prompt_pos'], max_sentnum, max_sentlen)
X_test_prompt_pos = pad_hierarchical_text_sequences(test_data['prompt_pos'], max_sentnum, max_sentlen)

X_train_prompt_pos = X_train_prompt_pos.reshape((X_train_prompt_pos.shape[0], X_train_prompt_pos.shape[1] * X_train_prompt_pos.shape[2]))
X_dev_prompt_pos = X_dev_prompt_pos.reshape((X_dev_prompt_pos.shape[0], X_dev_prompt_pos.shape[1] * X_dev_prompt_pos.shape[2]))
X_test_prompt_pos = X_test_prompt_pos.reshape((X_test_prompt_pos.shape[0], X_test_prompt_pos.shape[1] * X_test_prompt_pos.shape[2]))

X_train_linguistic_features = np.array(train_data['features_x'])
X_dev_linguistic_features = np.array(dev_data['features_x'])
X_test_linguistic_features = np.array(test_data['features_x'])

X_train_readability = np.array(train_data['readability_x'])
X_dev_readability = np.array(dev_data['readability_x'])
X_test_readability = np.array(test_data['readability_x'])

Y_train = np.array(train_data['y_scaled'])
Y_dev = np.array(dev_data['y_scaled'])
Y_test = np.array(test_data['y_scaled'])

X_train_attribute_rel = get_attribute_masks(Y_train)
X_dev_attribute_rel = get_attribute_masks(Y_dev)
X_test_attribute_rel = get_attribute_masks(Y_test)

print('================================')
print('X_train_pos: ', X_train_pos.shape)
print('X_train_prompt_words: ', X_train_prompt.shape)
print('X_train_prompt_pos: ', X_train_prompt_pos.shape)
print('X_train_readability: ', X_train_readability.shape)
print('X_train_ling: ', X_train_linguistic_features.shape)
print('X_train_attribute_rel: ', X_train_attribute_rel.shape)
print('Y_train: ', Y_train.shape)

print('================================')
print('X_dev_pos: ', X_dev_pos.shape)
print('X_dev_prompt_words: ', X_dev_prompt.shape)
print('X_dev_prompt_pos: ', X_dev_prompt_pos.shape)
print('X_dev_readability: ', X_dev_readability.shape)
print('X_dev_ling: ', X_dev_linguistic_features.shape)
print('X_dev_attribute_rel: ', X_dev_attribute_rel.shape)
print('Y_dev: ', Y_dev.shape)

print('================================')
print('X_test_pos: ', X_test_pos.shape)
print('X_test_prompt_words: ', X_test_prompt.shape)
print('X_test_prompt_pos: ', X_test_prompt_pos.shape)
print('X_test_readability: ', X_test_readability.shape)
print('X_test_ling: ', X_test_linguistic_features.shape)
print('X_test_attribute_rel: ', X_test_attribute_rel.shape)
print('Y_test: ', Y_test.shape)
print('================================')

max sent length: 50
max sent num: 97
max prompt sent length: 18
max prompt sent num: 8
X_train_pos:  (9513, 4850)
X_train_prompt_words:  (9513, 4850)
X_train_prompt_pos:  (9513, 4850)
X_train_readability:  (9513, 35)
X_train_ling:  (9513, 51)
X_train_attribute_rel:  (9513, 9)
Y_train:  (9513, 9)
X_dev_pos:  (1680, 4850)
X_dev_prompt_words:  (1680, 4850)
X_dev_prompt_pos:  (1680, 4850)
X_dev_readability:  (1680, 35)
X_dev_ling:  (1680, 51)
X_dev_attribute_rel:  (1680, 9)
Y_dev:  (1680, 9)
X_test_pos:  (1783, 4850)
X_test_prompt_words:  (1783, 4850)
X_test_prompt_pos:  (1783, 4850)
X_test_readability:  (1783, 35)
X_test_ling:  (1783, 51)
X_test_attribute_rel:  (1783, 9)
Y_test:  (1783, 9)


In [7]:
train_features_list = [X_train_pos, X_train_prompt, X_train_prompt_pos, X_train_linguistic_features, X_train_readability]
dev_features_list = [X_dev_pos, X_dev_prompt, X_dev_prompt_pos, X_dev_linguistic_features, X_dev_readability]
test_features_list = [X_test_pos, X_test_prompt, X_test_prompt_pos, X_test_linguistic_features, X_test_readability]

model = build_ProTACT(len(pos_vocab), len(word_vocab), max_sentnum, max_sentlen, 
                  X_train_readability.shape[1],
                  X_train_linguistic_features.shape[1],
                  configs, Y_train.shape[1], num_heads, embed_table)

evaluator = AllAttEvaluator(test_prompt_id, dev_data['prompt_ids'], test_data['prompt_ids'], dev_features_list,
                            test_features_list, Y_dev, Y_test, seed)

evaluator.evaluate(model, -1, print_info=True)

Attention x: (None, 46, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, 46, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 prompt_word_input (InputLa  [(None, 4850)]               0         []                            
 yer)                                    

                                                                                                  
 lstm (LSTM)                 (None, None, 100)            80400     ['multi_head_attention[0][0]']
                                                                                                  
 lstm_9 (LSTM)               (None, None, 100)            80400     ['multi_head_attention_9[0][0]
                                                                    ']                            
                                                                                                  
 lstm_1 (LSTM)               (None, None, 100)            80400     ['multi_head_attention_1[0][0]
                                                                    ']                            
                                                                                                  
 lstm_2 (LSTM)               (None, None, 100)            80400     ['multi_head_attention_2[0][0]
          

 lstm_12 (LSTM)              (None, None, 100)            80400     ['multi_head_attention_pe_2[0]
                                                                    [0]']                         
                                                                                                  
 lstm_13 (LSTM)              (None, None, 100)            80400     ['multi_head_attention_pe_3[0]
                                                                    [0]']                         
                                                                                                  
 lstm_14 (LSTM)              (None, None, 100)            80400     ['multi_head_attention_pe_4[0]
                                                                    [0]']                         
                                                                                                  
 lstm_15 (LSTM)              (None, None, 100)            80400     ['multi_head_attention_pe_5[0]
          

 reshape_1 (Reshape)         (None, 1, 186)               0         ['concatenate_1[0][0]']       
                                                                                                  
 reshape_2 (Reshape)         (None, 1, 186)               0         ['concatenate_2[0][0]']       
                                                                                                  
 reshape_3 (Reshape)         (None, 1, 186)               0         ['concatenate_3[0][0]']       
                                                                                                  
 reshape_4 (Reshape)         (None, 1, 186)               0         ['concatenate_4[0][0]']       
                                                                                                  
 reshape_5 (Reshape)         (None, 1, 186)               0         ['concatenate_5[0][0]']       
                                                                                                  
 reshape_6

                                                                    [0]']                         
                                                                                                  
 attention_22 (Attention)    (None, 1, 186)               0         ['tf.__operators__.getitem_1[0
                                                                    ][0]',                        
                                                                     'tf.compat.v1.boolean_mask_1[
                                                                    0][0]']                       
                                                                                                  
 attention_23 (Attention)    (None, 1, 186)               0         ['tf.__operators__.getitem_2[0
                                                                    ][0]',                        
                                                                     'tf.compat.v1.boolean_mask_2[
          

                                                                                                  
 flatten_3 (Flatten)         (None, 372)                  0         ['tf.concat_4[0][0]']         
                                                                                                  
 flatten_4 (Flatten)         (None, 372)                  0         ['tf.concat_5[0][0]']         
                                                                                                  
 flatten_5 (Flatten)         (None, 372)                  0         ['tf.concat_6[0][0]']         
                                                                                                  
 flatten_6 (Flatten)         (None, 372)                  0         ['tf.concat_7[0][0]']         
                                                                                                  
 flatten_7 (Flatten)         (None, 372)                  0         ['tf.concat_8[0][0]']         
          

KeyboardInterrupt: 

In [ ]:
# class CustomHistory(keras.callbacks.Callback):
#     def init(self):
#         self.train_loss = []
#         self.val_loss = []
#         self.train_acc = []
#         self.val_acc = []        
        
#     def on_epoch_end(self, batch, logs={}):
#         self.train_loss.append(logs.get('loss'))
#         self.val_loss.append(logs.get('val_loss'))
#         self.train_acc.append(logs.get('acc'))
#         self.val_acc.append(logs.get('val_acc'))
# custom_hist = CustomHistory()
# custom_hist.init() 

#  for ii in range(epochs):
#     print('Epoch %s/%s' % (str(ii + 1), epochs))
#     start_time = time.time()
#     model.fit(
#         train_features_list,
#         Y_train, batch_size=batch_size, epochs=5, verbose=0, shuffle=True, validation_data=(dev_features_list,Y_dev),callbacks=[custom_hist,checkpoint])
#     tt_time = time.time() - start_time
#     print("Training one epoch in %.3f s" % tt_time)
#     evaluator.evaluate(model, ii + 1)
#     print("Train Loss: ", custom_hist.train_loss[-1], "|| Val Loss: ", custom_hist.val_loss[-1])

# evaluator.print_final_info()

'''# show the loss as the graph
fig, loss_graph = plt.subplots()
loss_graph.plot(custom_hist.train_loss,'y',label='train loss')
loss_graph.plot(custom_hist.val_loss,'r',label='val loss')
loss_graph.set_xlabel('epoch')
loss_graph.set_ylabel('loss')
plt.savefig(str('images/protact/test_prompt_'+ str(test_prompt_id) + '_seed_' + str(seed) + '_loss.png'))'''

In [ ]:
# 저장한 체크포인트 있다면: 이어서 학습 시작할때 
# Checkpoint 폴더 안에 있는 .h5 파일 지울것
# model.load_weights('Checkpoint/tensor{epoch}')

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    # epoch 마다 파일명 다르게 저장
    filepath='Checkpoint/bestmodel{epoch}.h5',

    # epoch 마다 weights 들만 저장
    save_freq='epoch',
    save_weights_only = True,

    # validation accruary 가 최대일때만 저장 
    monitor='val_loss',
    mode='min'
)

In [8]:
class CustomHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = [] 
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        epoch_time = time.time() - self.start_time
        self.epoch_times.append(epoch_time)
        print(f"Epoch {epoch + 1}: Train Loss: {logs.get('loss')} || Val Loss: {logs.get('val_loss')}")
        print(f"Epoch {epoch + 1} completed in {epoch_time:.3f} seconds")

        # Evaluate the model (you might need to adjust this to your specific evaluation function)
        evaluator.evaluate(self.model, epoch + 1)

custom_hist = CustomHistory()
model.fit(
    train_features_list,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    shuffle=True,
    validation_data=(dev_features_list, Y_dev),
    callbacks=[custom_hist, checkpoint]
)

Epoch 1/50
trait num:  9
trait num:  9
952/952 [==============================] - ETA: 0s - loss: 0.0163trait num:  9
Epoch 1: Train Loss: 0.016327474266290665 || Val Loss: 0.013890832662582397
Epoch 1 completed in 262.809 seconds
56/56 [==============================] - 10s 183ms/step
CURRENT EPOCH: 1
[DEV] AVG QWK: 0.626
[DEV] score QWK: 0.733
[DEV] content QWK: 0.616
[DEV] organization QWK: 0.639
[DEV] word_choice QWK: 0.541
[DEV] sentence_fluency QWK: 0.554
[DEV] conventions QWK: 0.617
[DEV] prompt_adherence QWK: 0.645
[DEV] language QWK: 0.647
[DEV] narrativity QWK: 0.641
------------------------
[TEST] AVG QWK: 0.593
[TEST] score QWK: 0.818
[TEST] content QWK: 0.6
[TEST] organization QWK: 0.536
[TEST] word_choice QWK: 0.523
[TEST] sentence_fluency QWK: 0.563
[TEST] conventions QWK: 0.517
------------------------
[BEST TEST] AVG QWK: 0.593, {epoch}: 1
[BEST TEST] score QWK: 0.818
[BEST TEST] content QWK: 0.6
[BEST TEST] organization QWK: 0.536
[BEST TEST] word_choice QWK: 0.523
[B

KeyboardInterrupt: 

In [11]:
# 실행 X
# TEST: 위에서 진행된곳 까지 결과가 같다.
model.load_weights('Checkpoint/bestmodel1.h5')
evaluator.evaluate(model,1)

56/56 [==============================] - 10s 185ms/step
CURRENT EPOCH: 1
[DEV] AVG QWK: 0.626
[DEV] score QWK: 0.733
[DEV] content QWK: 0.616
[DEV] organization QWK: 0.639
[DEV] word_choice QWK: 0.541
[DEV] sentence_fluency QWK: 0.554
[DEV] conventions QWK: 0.617
[DEV] prompt_adherence QWK: 0.645
[DEV] language QWK: 0.647
[DEV] narrativity QWK: 0.641
------------------------
[TEST] AVG QWK: 0.593
[TEST] score QWK: 0.818
[TEST] content QWK: 0.6
[TEST] organization QWK: 0.536
[TEST] word_choice QWK: 0.523
[TEST] sentence_fluency QWK: 0.563
[TEST] conventions QWK: 0.517
------------------------
[BEST TEST] AVG QWK: 0.593, {epoch}: 1
[BEST TEST] score QWK: 0.818
[BEST TEST] content QWK: 0.6
[BEST TEST] organization QWK: 0.536
[BEST TEST] word_choice QWK: 0.523
[BEST TEST] sentence_fluency QWK: 0.563
[BEST TEST] conventions QWK: 0.517
--------------------------------------------------------------------------------------------------------------------------
